In [2]:
from huggingface_hub import login
login()

In [2]:
from datasets import load_dataset

ds = load_dataset("ruslanmv/ai-medical-chatbot", split="train")

In [4]:
ds

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 256916
})

In [5]:
ds[0]

{'Description': 'Q. What does abutment of the nerve root mean?',
 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?',
 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}

In [3]:
import pandas as pd

df = ds.to_pandas()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256916 entries, 0 to 256915
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Description  256916 non-null  object
 1   Patient      256916 non-null  object
 2   Doctor       256916 non-null  object
dtypes: object(3)
memory usage: 5.9+ MB


In [9]:
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [15]:
df[df.duplicated(subset='Description')]

,Description,Patient,Doctor
16542,Q. Why have I not got my periods yet?,"Hello doctor, My fiancee got her periods one a...",Hi. Mostly pill is effective if taken within 7...
26693,Q. Is it possible to get pregnant without pene...,"Hello doctor, I and my fiancee messed up. Beca...","Hello. Well, in this case I see that her guilt..."
26940,Q. How can I gain weight?,"Hello doctor, I want to gain weight. Please help.",Hello. I just read your query. You did not men...
27505,Q. What is the treatment for chalazion?,"Hello doctor,I have a red bump under my low ey...",Hello. You are right it is a chalazion (attach...
27772,Q. What is the cause for increased bilirubin l...,"Hello doctor,I am a 50-year-old male, slim, an...",Hello. You do not have any complaints. 1. Tota...
...,...,...,...
256882,Suggest better treatment for hair fall,I am a girl......23 years old......My height i...,"Hello, I have gone through your query and woul..."
256884,Suggest treatment for alopecia areata,"hello sir,i am suffering from alopacia areata,...","Hello, I have gone through your query and intr..."
256885,Suggest treatment for hair fall,doctor please i need ur help.im 26 years old f...,"Hello, I have gone through your query and woul..."
256892,What causes hair fall and thinning?,My Endo prescribed Armour in December. It s be...,"Hello, I have gone through your query and woul..."


In [4]:
df = df.drop_duplicates(subset=['Description', 'Patient'], )

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246510 entries, 0 to 256915
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Description  246510 non-null  object
 1   Patient      246510 non-null  object
 2   Doctor       246510 non-null  object
dtypes: object(3)
memory usage: 7.5+ MB


In [23]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-12 09:43:46--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-12 09:43:46 (46.0 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [19]:
import minsearch

In [20]:
index = minsearch.Index(
    text_fields=['Description','Patient', 'Doctor'],
    keyword_fields=[""]
)

In [11]:
documents = df.to_dict(orient='records')

In [22]:
index.fit(documents)

In [23]:
def search(query):
    boost = {'Description': 2}

    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=10
    )

    return results

In [24]:
search('I have a headache and low fever. What should I do?')

[{'Description': 'What could cause low grade fever and headache?',
  'Patient': 'My niece is 4 yrs old and has a low grade fever and a headache. She has been to the doctor and is on amoxicillin. She started out with a headache, then a low grade fever that escalated to 105 and vomiting. My concern is she has been on antibiotics for 2 days, she was tested for strep throat and that was negative. What could be causing the headache?',
  'Doctor': 'Hi, Does she have abdominal pain or loose stools? Fever, headache and vomiting is most likely from a viral fever. Let her take some rest, take paracetamol 3 times daily even if afebrile(for headache) and drink plenty of fluids in small frequent sips. This will subside in 3-4 days. Nothing to worry if she is otherwise playful and taking feeds. Get back to doctor if fever does not subside by then, headache is sever or vomiting persists. Hope I have answered your query. Let me know if I can assist you further. Take care Regards, Dr. Muhammed Aslam TK

In [5]:
from openai import OpenAI

client = OpenAI()

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You are a knowledgeable and empathetic medical assistant specializing in providing detailed and accurate information based on real patient-doctor conversations. 
You must interpret the given context to address the user's question effectively.
Your responses should be clear, concise, and consider the medical and emotional aspects of the situation. 
Use the context provided to ensure that your answer is grounded in the information retrieved, avoiding speculation or providing advice outside of the context.

Question: {question}

Context:
{context}

Instructions:
1. Analyze the context carefully to identify relevant information related to the user's question.
2. Provide an informative and empathetic response, ensuring it aligns with the retrieved context.
3. If the context does not directly answer the question, use general medical knowledge cautiously to give a helpful but non-definitive response.
4. Prioritize the clarity and accuracy of information, using language accessible to a general audience.
5. If necessary, indicate when it would be appropriate for the user to seek further medical consultation.
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Description: {doc['Description']}\nPatient: {doc['Patient']}\nDoctor: {doc['Doctor']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [28]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
rag('Hi Doctor, I want to know the symptoms of sinusitis. I think I have it.')

'Hi there,\n\nThe symptoms of sinusitis typically include:\n\n1. **Facial Pain and Pressure**: You might feel pain and pressure around your forehead, eyes, nose, and cheeks.\n2. **Nasal Congestion**: This can lead to difficulty breathing through your nose.\n3. **Thick Nasal Discharge**: The mucus may be yellow or green.\n4. **Loss of Smell**: Your sense of smell might be reduced or lost.\n5. **Cough**: This can be worse at night.\n6. **Sore Throat**: Post-nasal drip can cause throat irritation.\n7. **Headache**: Pain associated with sinusitis can radiate to the head, leading to headaches.\n8. **Ear Pain or Fullness**: Sensation of fullness or pain in the ear.\n9. **Fatigue**: Feeling unusually tired or exhausted.\n10. **Bad Breath**: Often due to the accumulation of bacteria in the sinuses.\n\nIn some conversations, patients reported symptoms like pain in the head, yellow mucus, and stuffiness which align with sinusitis. If you are experiencing any of these symptoms, it might be helpfu

In [31]:
results = search('Hi Doctor, I want to know the symptoms of sinusitis. I think I have it.')

In [1]:
from elasticsearch import Elasticsearch

In [8]:
es_client = Elasticsearch('http://localhost:9200')

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Description": {"type": "text"},
            "Patient": {"type": "text"},
            "Doctor": {"type": "text"}
        }
    }
}

index_name = "health_questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'health_questions'})

In [10]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/246510 [00:00<?, ?it/s]

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

## Retrieval evaluation

In [ ]:
import pandas as pd

In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
df = 